<a href="https://colab.research.google.com/github/jessica-guan/Python-DataSci-ML/blob/main/Natural%20Language%20Processing%3A%20Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 21: Natural Language Processing II**
---

### **Description**
In today's lab, we have another text classification task, but this time we will be using **embeddings.** For this project, we will be working with a dataset of BBC News articles classified by topic.

<br>

### **Lab Structure**

**Part 1**: [Text Classification of BBC Articles](#p1)

**Part 2**: [Convolutional Neural Networks](#p2)


**Part 3**: [[OPTIONAL] IMDB Sentiment Classification](#p3)

<br>

### **Goals**
By the end of this lab, you will:
* Understand how to apply embedding layers in models.
* Compare a fully connected network to a CNN for text classification with embeddings.

<br>

### **Cheat Sheets**
[Natural Language Processing II](https://docs.google.com/document/d/1p3xVUL1F6SEkusCI4klPLYqQwCkVN5s00ZvJjBpiSqM/edit?usp=sharing)

<br>

**Before starting, run the code below to import all necessary functions and libraries.**

In [ ]:
!pip install lime

from lime import lime_text
import numpy as np
import pandas as pd

import tensorflow as tf
import numpy as np
import os

from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

from random import choices

import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=a410cf3425f24438edabfbfb2f224c6ee64b827827ccd787da183a8172564bc3
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


<a name="p1"></a>

---
## **Part 1: Text Classification of BBC Articles**
---

In this section, we'll apply our text classification knowledge to a corpus of BBC articles. Your task is to develop a model that categorizes articles based on snippets of text, assigning each to a specific category. Unlike previous labs where we focused on visual data, here we'll use neural networks with traditional Dense layers to process and classify text data.

<br>



**Run the code provided below to import the dataset.**

In [ ]:
dataset = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRRiQ1DUkUxk31YpaHA2i9QtwGq_VGXiy86z7l3aT9v5zoB6M7a-2M2qlYckr1C_ZG6StBELlU_hD3S/pub?output=csv')

### **Problem #1.1: Determine the number of categories**


Using any necessary pandas functions or attributes, determine the total number of unique categories that the texts are assigned to.

**In the cell code below, display the first few rows in the dataset.**


In [ ]:
dataset.head()

,text,category,category_id
0,libya takes 1bn unfrozen funds libya withdrawn...,business,0
1,saudi investor picks savoy famous savoy hotel ...,business,0
2,tate lyle boss bags award tate lyle chief exec...,business,0
3,uk economy facing major risks uk manufacturing...,business,0
4,aids climate davos agenda climate change fight...,business,0


Let's now see which of the categories in our dataset are unique.

In [ ]:
print(dataset["category"].unique())
print(len(dataset["category"].unique()))

['business' 'entertainment' 'politics' 'sport' 'tech']
5


**Question:** How many unique categories do we have from the above output? What are the unique categories?

### **Problem #1.2: Split the data into training and test sets**


Determine the correct variables to use as the feature(s) and label here, making sure to provide numerical labels for the neural network to predict.

In [ ]:
# Split the dataset into features and labels
x = dataset['text'].values
y = dataset['category_id'].values

# Split the dataset into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### **Problem #1.3: Create the `TextVectorization` layer**


To get started building the neural network, create a `TextVectorization` layer to vectorize this data.

Specifically,
1. Initialize the layer with the specified parameters.

2. Adapt the layer to the training data.

3. Look at the newly built vocabulary.

#### **1.Initialize the layer with the specified parameters.**

* The vocabulary should be at most 10000 words.
* The layer's output should always be 20 integers.

In [ ]:
vectorize_layer = TextVectorization(
    max_tokens = 10000,
    output_mode = 'int',
    output_sequence_length = 20
  )

#### **2. Adapt the layer to the training data.**

In [ ]:
vectorize_layer.adapt(x_train)

#### **3. Look at the newly built vocabulary.**

In [ ]:
vectorize_layer.get_vocabulary()[:50]

['',
 '[UNK]',
 'mr',
 'm',
 'people',
 'new',
 '£',
 't',
 'government',
 'film',
 'year',
 'uk',
 'music',
 'game',
 'world',
 'best',
 'labour',
 'election',
 'time',
 'blair',
 '1',
 'party',
 'games',
 'mobile',
 'market',
 'england',
 '000',
 'tv',
 '3',
 '2',
 'sales',
 'like',
 'years',
 'told',
 'home',
 'million',
 'company',
 'bbc',
 'says',
 'play',
 'technology',
 '6',
 'tax',
 'net',
 'firm',
 'win',
 'economy',
 'digital',
 'brown',
 'number']

### **Problem #1.4: Build the model**


Complete the code below to build a model with the following layers.

An `Embedding` layer such that:
* The vocabulary contains 10000 tokens.
* The input length corresponds to the output of the vectorization layer.
* The number of outputs per input is 200.

<br>

Two `Dense` layers with a number of neurons and activation function that you choose. We recommend you try a few options.

<br>

A `Dense` layer for outputting classification probabilities for each of the possible categories.


*Hint: If you're not sure which activation function to use, use `relu`.*

In [ ]:
model = Sequential()

# Input, Vectorization, and Embedding Layers
model.add(Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim=10000, output_dim=200))

# Hidden Layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))

# Output Layer
model.add(Dense(5, activation='softmax'))

### **Problem #1.5: Compile and fit the model**

Using standard parameters for classification, compile and train 8yncthis neural network using:
* A learning rate of 0.01.
* A batch size of 200.
* 5 epochs.

In [ ]:
opt = Adam(learning_rate = 0.01)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=200, epochs=5)

Epoch 1/5
9/9 [==============================] - 5s 187ms/step - loss: 1.1841 - accuracy: 0.5292
Epoch 2/5
9/9 [==============================] - 2s 185ms/step - loss: 0.0324 - accuracy: 0.9938
Epoch 3/5
9/9 [==============================] - 2s 165ms/step - loss: 0.0065 - accuracy: 0.9983
Epoch 4/5
9/9 [==============================] - 2s 170ms/step - loss: 0.0279 - accuracy: 0.9961
Epoch 5/5
9/9 [==============================] - 2s 191ms/step - loss: 0.0335 - accuracy: 0.9978


### **Problem #1.6: Evaluate the model**


Now, evaluate the model for both the training and test sets.

In [ ]:
# Evaluate the model on the training set
train_loss, train_accuracy = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy:", train_accuracy)
print("Training Loss:", train_loss)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy:", test_accuracy)
print("Test Loss:", test_loss)

Training Accuracy: 1.0
Training Loss: 1.4173429008224048e-05
Test Accuracy: 0.9123595356941223
Test Loss: 1.3674368858337402


<a name="p2"></a>

---
## **Part 2:  Convolutional Neural Networks**
---

In this section, you will classify the articles using neural networks with `Conv1D` and `MaxPooling1D` hidden layers. Feel free to include `Dense` hidden layers too.

### **Problem #2.1: Create the highest performing model possible using CNNs**


Complete the code below to train a new model that is identical to the one above, except using any or all of the CNN layers that keras provides. The goal is to create a model that performs as well as possible on the *test set*.

In [ ]:
model = Sequential()

# Input, Vectorization, and Embedding Layers
model.add(Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim=2000, output_dim=128))
model.add(Conv1D(filters=63, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Hidden Layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))

# Output Layer
model.add(Dense(5, activation='softmax'))

# Printing Structure
for layer in model.layers:
  print(str(layer.input_shape) + " -> " + str(layer.output_shape))
print("\n\n\n")

# Fitting
opt = Adam(learning_rate = 0.01)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=200, epochs=5)

# Evaluating
print("\n\n\n")
model.evaluate(x_train, y_train, verbose=0)
model.evaluate(x_test, y_test, verbose=0)

(None, 1) -> (None, 20)
(None, 20) -> (None, 20, 128)
(None, 20, 128) -> (None, 18, 63)
(None, 18, 63) -> (None, 9, 63)
(None, 9, 63) -> (None, 567)
(None, 567) -> (None, 64)
(None, 64) -> (None, 5)




Epoch 1/5
9/9 [==============================] - 3s 186ms/step - loss: 1.3978 - accuracy: 0.3815
Epoch 2/5
9/9 [==============================] - 1s 152ms/step - loss: 0.2268 - accuracy: 0.9388
Epoch 3/5
9/9 [==============================] - 1s 111ms/step - loss: 0.0366 - accuracy: 0.9904
Epoch 4/5
9/9 [==============================] - 1s 102ms/step - loss: 0.0186 - accuracy: 0.9972
Epoch 5/5
9/9 [==============================] - 1s 135ms/step - loss: 0.0290 - accuracy: 0.9949






[1.267504334449768, 0.8853932619094849]

<a name="p3"></a>

---
## **Part 3: [OPTIONAL] IMDB Sentiment Classification**
---

In this part we will focus on building a CNN model using the IMDB sentiment classification dataset. This is a dataset of 25,000 movie reviews with sentiment labels: 0 for negative and 1 for positive.

<br>


**Run the code provided below to import the dataset.**

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTdgncgNHtppfS89LHOh1kGl5tYzoEUrUwmOPOQF7mQ0U5Rzba27H45imvZ06_J2x0-wCJySylP5V3_/pub?gid=1712575053&single=true&output=csv'

df = pd.read_csv(url)
df.head()

x_train, x_test, y_train, y_test = train_test_split(df["review"], df["sentiment"], test_size = 0.2, random_state = 42)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### **Problem #3.1: Create the `TextVectorization` layer**


To get started, let's create a `TextVectorization` layer to vectorize this data.

Specifically,
1. Initialize the layer with the specified parameters.

2. Adapt the layer to the training data.

3. Look at the newly built vocabulary.

#### **1. Initialize the layer with the specified parameters.**

* The vocabulary should be at most 5000 words.
* The layer's output should always be 64 integers.

In [ ]:
vectorize_layer = TextVectorization(
    max_tokens = 5000,
    output_mode = 'int',
    output_sequence_length = 64
  )

#### **2. Adapt the layer to the training data.**

In [ ]:
vectorize_layer.adapt(x_train)

#### **3. Look at the newly built vocabulary.**

In [ ]:
vectorize_layer.get_vocabulary()[:50]

['',
 '[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'this',
 'that',
 'br',
 'was',
 'as',
 'for',
 'with',
 'movie',
 'but',
 'film',
 'on',
 'not',
 'you',
 'are',
 'his',
 'have',
 'be',
 'he',
 'one',
 'its',
 'at',
 'all',
 'by',
 'an',
 'they',
 'from',
 'who',
 'so',
 'like',
 'just',
 'or',
 'her',
 'about',
 'if',
 'has',
 'out',
 'some',
 'there',
 'what']

### **Problem #3.2: Build and Train a Dense model**

Complete the code below to build a model with the following layers.

An Embedding layer such that:
- The vocabulary contains 5000 tokens.
- The input length corresponds to the output of the vectorization layer.
- The number of outputs per input is 128.

<br>

Hidden layers such that:

- There's at least one Dense layer.

<br>

A Dense layer for outputting classification probabilities for "negative" or "positive" labels.

In [ ]:
model = Sequential()

# Input, Vectorization, and Embedding Layers
model.add(Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim=5000, output_dim=128))
model.add(Conv1D(filters=63, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Hidden Layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))

# Output Layer
model.add(Dense(2, activation='softmax'))

# Printing Structure
for layer in model.layers:
  print(str(layer.input_shape) + " -> " + str(layer.output_shape))
print("\n\n\n")

# Fitting
opt = Adam(learning_rate = 0.01)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=200, epochs=5)

# Evaluating
print("\n\n\n")
model.evaluate(x_train, y_train, verbose=0)
model.evaluate(x_test, y_test, verbose=0)

(None, 1) -> (None, 64)
(None, 64) -> (None, 64, 128)
(None, 64, 128) -> (None, 62, 63)
(None, 62, 63) -> (None, 31, 63)
(None, 31, 63) -> (None, 1953)
(None, 1953) -> (None, 64)
(None, 64) -> (None, 2)




Epoch 1/5
200/200 [==============================] - 17s 75ms/step - loss: 0.5824 - accuracy: 0.6863
Epoch 2/5
200/200 [==============================] - 6s 30ms/step - loss: 0.4333 - accuracy: 0.7992
Epoch 3/5
200/200 [==============================] - 6s 30ms/step - loss: 0.3339 - accuracy: 0.8586
Epoch 4/5
200/200 [==============================] - 4s 22ms/step - loss: 0.2182 - accuracy: 0.9158
Epoch 5/5
200/200 [==============================] - 7s 34ms/step - loss: 0.1271 - accuracy: 0.9581






[0.7545827627182007, 0.7677000164985657]

This other alternative includes building the model with CNN.
**Which architecture performs better?**

In [ ]:
# [OPTIONAL] USING CNNs
model = Sequential()

# Input, Vectorization, and Embedding Layers
model.add(Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim = 5000, output_dim = 128, input_length = 64))

# Hidden Layers
model.add(Conv1D(filters = 16, kernel_size = 4, activation = 'relu'))
model.add(MaxPooling1D(pool_size = 3))
model.add(Flatten())

# Output Layer
model.add(Dense(2, activation = 'softmax'))



# Printing Structure
for layer in model.layers:
  print(str(layer.input_shape) + " -> " + str(layer.output_shape))
print("\n\n\n")



# Fitting
opt = Adam(learning_rate = 0.001)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, epochs = 5, batch_size = 256)


# Evaluating
print("\n\n\n")
model.evaluate(x_train, y_train)
model.evaluate(x_test, y_test)

(None, 1) -> (None, 64)
(None, 64) -> (None, 64, 128)
(None, 64, 128) -> (None, 61, 16)
(None, 61, 16) -> (None, 20, 16)
(None, 20, 16) -> (None, 320)
(None, 320) -> (None, 2)




Epoch 1/5
157/157 [==============================] - 15s 85ms/step - loss: 0.5821 - accuracy: 0.6798
Epoch 2/5
157/157 [==============================] - 8s 48ms/step - loss: 0.3865 - accuracy: 0.8257
Epoch 3/5
157/157 [==============================] - 4s 28ms/step - loss: 0.3193 - accuracy: 0.8640
Epoch 4/5
157/157 [==============================] - 6s 36ms/step - loss: 0.2461 - accuracy: 0.9081
Epoch 5/5
157/157 [==============================] - 4s 24ms/step - loss: 0.1599 - accuracy: 0.9539




313/313 [==============================] - 2s 5ms/step - loss: 0.5091 - accuracy: 0.7856


[0.5091047286987305, 0.7856000065803528]

---
###© 2024 The Coding School, All rights reserved